In [5]:
import numpy as np
import pandas as pd
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
def read_json_to_dataframe(filepath, test_file=False):
    with open(filepath) as f:
        data = json.load(f)
        
    df = pd.DataFrame(data)
    df['url'] = df.images.map(lambda x: x['url'][0])
    if not test_file:
        df['image_id'] = df.annotations.map(lambda x: x['image_id'])
        df['label_id'] = df.annotations.map(lambda x: x['label_id'])
        df.drop(columns=['annotations', 'images'], inplace=True)
    return df

In [9]:
train_data = read_json_to_dataframe('train.json', test_file=False)
validation_data = read_json_to_dataframe('validation.json', test_file=False)
test_data = read_json_to_dataframe('test.json', test_file=True)

In [10]:
print("Train size: ", train_data.shape)
print("Validation size: ", validation_data.shape)
print("Test size: ", test_data.shape)

('Train size: ', (194828, 3))
('Validation size: ', (6400, 3))
('Test size: ', (12800, 2))


In [11]:
tfidf = TfidfVectorizer(analyzer='char', ngram_range=(1, 3), max_features=2500, lowercase=False)
tfidf.fit(train_data.url)

TfidfVectorizer(analyzer='char', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=False, max_df=1.0, max_features=2500, min_df=1,
        ngram_range=(1, 3), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [12]:
train_features = tfidf.transform(train_data.url)
validation_features = tfidf.transform(validation_data.url)
test_features = tfidf.transform(test_data.url)

In [14]:
from sklearn.decomposition import TruncatedSVD
tsvd = TruncatedSVD(n_components=100)
X_train_pca = tsvd.fit_transform(train_features)
X_valid_pca = tsvd.fit_transform(validation_features)
X_test_pca = tsvd.transform(test_features)

In [15]:
clf = LogisticRegression()
clf.fit(X_train_pca,train_data.label_id)


NameError: name 'y_test' is not defined

In [17]:
submission = pd.DataFrame({
    'id': 1 + np.arange(len(test_data)),
    'predicted': clf.predict(X_test_pca),
})
submission.head()

,id,predicted
0,1,125
1,2,42
2,3,103
3,4,21
4,5,116


In [18]:
submission.to_csv('submission.csv', index=False)